In [9]:
from dotenv import load_dotenv
load_dotenv()
import os
import yaml
from crewai import Agent, Task, Crew,LLM
ollama_llm = LLM(
            model="ollama/llama3.2:latest",
            #model="ollama/gemma3:4b",
            api_base="http://localhost:11434"
            )

### read configs

In [10]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [11]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

### Creating Agents and Tasks

In [12]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
  llm =  ollama_llm,
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
  llm =  ollama_llm,
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
  llm =  ollama_llm,
)

In [13]:
# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

In [14]:
# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

### Crew's Input

In [15]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



### Kicking off the crew

In [16]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d6a59a73-fe9c-427b-a8ad-6afa93dc0a10                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 19e2066f-c78d-4444-9efe-a813e1a7609c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 19e2066f-c78d-4444-9efe-a813e1a7609c
       Status: Executing Task...
    └── 🤖 Agent: The Ultimate Project Planner
        
            Status: In Progress

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

🚀 Crew: crew
└── 📋 Task: 19e2066f-c78d-4444-9efe-a813e1a7609c
       Status: Executing Task...
    └── 🤖 Agent: The Ultimate Project Planner
        
            Status: ✅ Completed

...

# Agent: Resource Allocation Strategist
## Task: Strategically allocate tasks for the Website project to team members based on their skills, availability, and current workload. Ensure that each task is assigned to the most suitable team member and that the workload is evenly distributed.

Team members:

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)




# Agent: Resource Allocation Strategist
## Final Answer: 
{
  "tasks": [
    {
      "task_name": "Create a responsive design that works well on desktop and mobile devices",
      "estimated_time_hours": 40,
      "required_resources": ["Bob Smith (Designer)", "John Doe (Project Manager)"]
    },
    {
      "task_name": "Implement a modern, visually appealing user interface with a clean look",
      "estimated_time_hours": 60,
      "required_resources": ["Jane Doe (Software Engineer)", "Bob Smith (Designer)"]
    },
    {
      "task_name": "De

🚀 Crew: crew
├── 📋 Task: 19e2066f-c78d-4444-9efe-a813e1a7609c
│      Assigned to: The Ultimate Project Planner
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Ultimate Project Planner
│       
│           Status: ✅ Completed
├── 📋 Task: de2c87de-9237-42bf-b34c-62bbb05c3531
│      Assigned to: Expert Estimation Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Estimation Analyst
│       
│           Status: ✅ Completed
└── 📋 Task: b578aeed-56ca-4faf-9fba-5e5f2e4524bc
       Status: Executing Task...
    └── 🤖 Agent: Resource Allocation Strategist
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 19e2066f-c78d-4444-9efe-a813e1a7609c
│      Assigned to: The Ultimate Project Planner
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Ultimate Project Planner
│       
│           Status: ✅ Completed
├── 📋 Task: de2c87de-9237-42bf-b34c-62bbb05c3531
│      Assigned to: Expert Estimation Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Estimation Analyst
│       
│           Status: ✅ Completed
└── 📋 Task: b578aeed-56ca-4faf-9fba-5e5f2e4524bc
       Assigned to: Resource Allocation Strategist
    
       Status: ✅ Completed
    └── 🤖 Agent: Resource Allocation Strategist
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b578aeed-56ca-4faf-9fba-5e5f2e4524bc                                                                     │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d6a59a73-fe9c-427b-a8ad-6afa93dc0a10                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Usage Metrics and Costs

In [17]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0015


/var/folders/xd/3z5vvpds0zxf_pypxd4cn3d80000gn/T/ipykernel_62862/1074044196.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,10111,6217,0,3894,3


### Result

In [21]:
print(result.raw)

{
  "tasks": [
    {
      "task_name": "Create a responsive design that works well on desktop and mobile devices",
      "estimated_time_hours": 40,
      "required_resources": ["Bob Smith (Designer)", "John Doe (Project Manager)"]
    },
    {
      "task_name": "Implement a modern, visually appealing user interface with a clean look",
      "estimated_time_hours": 60,
      "required_resources": ["Jane Doe (Software Engineer)", "Bob Smith (Designer)"]
    },
    {
      "task_name": "Develop a user-friendly navigation system with intuitive menu structure",
      "estimated_time_hours": 40,
      "required_resources": ["John Doe (Project Manager)", "Jane Doe (Software Engineer)"]
    },
    {
      "task_name": "Include an 'About Us' page highlighting the company's history and values",
      "estimated_time_hours": 30,
      "required_resources": ["Alice Johnson (QA Engineer)", "Bob Smith (Designer)"]
    },
    {
      "task_name": "Design a 'Services' page showcasing the business's

In [22]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

/var/folders/xd/3z5vvpds0zxf_pypxd4cn3d80000gn/T/ipykernel_62862/793840085.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  tasks = result.pydantic.dict()['tasks']


,task_name,estimated_time_hours,required_resources
0,Create a responsive design that works well on desktop and mobile devices,40.000000,"['Bob Smith (Designer)', 'John Doe (Project Manager)']"
1,"Implement a modern, visually appealing user interface with a clean look",60.000000,"['Jane Doe (Software Engineer)', 'Bob Smith (Designer)']"
2,Develop a user-friendly navigation system with intuitive menu structure,40.000000,"['John Doe (Project Manager)', 'Jane Doe (Software Engineer)']"
3,Include an 'About Us' page highlighting the company's history and values,30.000000,"['Alice Johnson (QA Engineer)', 'Bob Smith (Designer)']"
4,Design a 'Services' page showcasing the business's offerings with descriptions,40.000000,"['Bob Smith (Designer)', 'Jane Doe (Software Engineer)']"
5,Create a 'Contact Us' page with a form and integrated map for communication,40.000000,"['Tom Brown (QA Engineer)', 'John Doe (Project Manager)']"
6,Implement a blog section for sharing industry news and company updates,60.000000,"['Jane Doe (Software Engineer)', 'Bob Smith (Designer)']"
7,Ensure fast loading times and optimize for search engines (SEO),40.000000,"['Alice Johnson (QA Engineer)', 'John Doe (Project Manager)']"
8,Integrate social media links and sharing capabilities,20.000000,"['Bob Smith (Designer)', 'Tom Brown (QA Engineer)']"
9,Include a testimonials section to showcase customer feedback and build trust,30.000000,"['Alice Johnson (QA Engineer)', 'Jane Doe (Software Engineer)']"


### Inspecting Milestones

In [23]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

/var/folders/xd/3z5vvpds0zxf_pypxd4cn3d80000gn/T/ipykernel_62862/795124587.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  milestones = result.pydantic.dict()['milestones']


,milestone_name,tasks
0,Task 1 Completion,['Create a responsive design that works well on desktop and mobile devices']
1,Task 2 Completion,"['Implement a modern, visually appealing user interface with a clean look']"
2,Task 3 Completion,['Develop a user-friendly navigation system with intuitive menu structure']
3,Task 4 Completion,"[""Include an 'About Us' page highlighting the company's history and values""]"
4,Task 5 Completion,"[""Design a 'Services' page showcasing the business's offerings with descriptions""]"
5,Task 6 Completion,"[""Create a 'Contact Us' page with a form and integrated map for communication""]"
6,Task 7 Completion,['Implement a blog section for sharing industry news and company updates']
7,Task 8 Completion,['Ensure fast loading times and optimize for search engines (SEO)']
8,Task 9 Completion,['Integrate social media links and sharing capabilities']
9,Task 10 Completion,['Include a testimonials section to showcase customer feedback and build trust']
